In [4]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 50.85,
	"longitude": 4.35,
	"start_date": "1995-06-20",
	"end_date": "1995-06-20",
	"hourly": ["temperature_2m", "rain", "wind_speed_10m"],
	"timezone": "Europe/Berlin",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_rain = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time() + response.UtcOffsetSeconds(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd() + response.UtcOffsetSeconds(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["rain"] = hourly_rain
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)

Coordinates: 50.86115646362305°N 4.331550598144531°E
Elevation: 27.0 m asl
Timezone: b'Europe/Berlin'b'GMT+1'
Timezone difference to GMT+0: 3600s

Hourly data
                         date  temperature_2m  rain  wind_speed_10m
0  1995-06-20 00:00:00+00:00       14.580000   0.0        7.172949
1  1995-06-20 01:00:00+00:00       14.130000   0.0        7.172949
2  1995-06-20 02:00:00+00:00       13.580000   0.0        6.989935
3  1995-06-20 03:00:00+00:00       13.030000   0.0        6.877790
4  1995-06-20 04:00:00+00:00       12.530000   0.0        7.208994
5  1995-06-20 05:00:00+00:00       12.280000   0.0        7.568566
6  1995-06-20 06:00:00+00:00       13.130000   0.0        8.647496
7  1995-06-20 07:00:00+00:00       15.730000   0.0        7.559999
8  1995-06-20 08:00:00+00:00       18.080000   0.0        6.489992
9  1995-06-20 09:00:00+00:00       19.679998   0.0        6.151683
10 1995-06-20 10:00:00+00:00       20.879999   0.0        6.109403
11 1995-06-20 11:00:00+00:00       2